In [1]:
import webbrowser
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
from sklearn.metrics import mean_squared_error,r2_score
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
apps_df=pd.read_csv('play_store_data.csv')
review_df=pd.read_csv('user_reviews.csv')

In [5]:
apps_df.head(5)

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
review_df.head(5)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [7]:
apps_df=apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df=apps_df[apps_df['Rating']<=5]
review_df.dropna(subset=['Translated_Review'],inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\337591803.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\337591803.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [8]:
apps_df.dtypes

Unnamed: 0          int64
App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [9]:
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)



In [10]:
apps_df.dtypes

Unnamed: 0          int64
App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [11]:
merge_df=pd.merge(apps_df,review_df,on='App',how='inner')

In [12]:
merge_df.head(5)

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


In [13]:
#rating column
def rating_group(rating):
    if rating>=1 and rating<2:
        return '1-2'
    elif rating>=2 and rating<3:
        return '2-3'
    elif rating>=3 and rating<4:
        return '3-4'
    else:
        return '4-5'
merge_df['Group of Rating']=merge_df['Rating'].apply(rating_group)


In [14]:
apps_df['Revenue']=apps_df['Installs']*apps_df['Price']

In [15]:
sia=SentimentIntensityAnalyzer()

In [16]:
merge_df['Sentiment Score']=merge_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [17]:
merge_df['Last Updated']=pd.to_datetime(merge_df['Last Updated'],errors='coerce')

In [18]:
merge_df['Year']=merge_df['Last Updated'].dt.year

In [19]:
merge_df.head()

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,...,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Group of Rating,Sentiment Score,Year
0,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,...,2018-01-15,2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000,3-4,-0.2500,2018
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,...,2018-01-15,2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333,3-4,-0.8020,2018
2,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,...,2018-01-15,2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000,3-4,0.3612,2018
3,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,...,2018-01-15,2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000,3-4,0.6369,2018
4,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,...,2018-01-15,2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000,3-4,-0.5719,2018


In [20]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [21]:
plot_containers=""

In [22]:
def save_plot_as_html(fig,filename,insight):
    global plot_containers
    filepath=os.path.join(html_files_path,filename)
    html_content= pio.to_html(fig,full_html=False,include_plotlyjs='inline')
    plot_containers+= f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot"> {html_content}</div> 
        <div class="insights">{insight}<div> 
    </div>
    """
    fig.write_html(filepath,full_html=False,include_plotlyjs='inline')

In [23]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [24]:
#fig1 sentiment score(bar)
filtered_df=merge_df[merge_df['Reviews']>1000].dropna(subset=['Reviews'])	
top_5_categories = filtered_df['Category'].value_counts().nlargest(5).index
filtered_df = filtered_df[filtered_df['Category'].isin(top_5_categories)]
sentiment_counts = filtered_df.groupby(['Category', 'Group of Rating', 'Sentiment Score']).size().reset_index(name='count')

fig = px.bar(sentiment_counts,
                 x='Category',
                 y='count',
                 color='Sentiment Score',
                 facet_col='Group of Rating',
                 category_orders={'Group of Rating': ['1-2', '2-3', '3-4','4-5']},
                 title='Sentiment Distribution by Category and Rating Group',
                 labels={'count': 'Number of Reviews', 'Sentiment': 'Sentiment Score'}
            )

fig.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=8,r=8,t=30,b=8)
)
fig.update_traces(marker=dict(line=dict(color='white',width=4)))  
save_plot_as_html(fig,"Sentiment Score Graph1 new .html", "Sentiment Analysis on the Playstore")

In [30]:
import plotly.graph_objects as go
import pandas as pd
import datetime
import pytz

In [31]:
data = {
    'Country': ['USA', 'UK',  'India', 'Brazil', 'China', 'Australia', 'Nigeria'],
    }
country_df = pd.DataFrame(data)
apps_df=pd.read_csv('play_store_data.csv')
merge_df=pd.merge(apps_df,country_df,how='cross')
merge_df['Installs']= pd.to_numeric(merge_df['Installs'],errors='coerce')

In [42]:
def is_within_time_range():
    """Checks if the current time is within the specified IST time range."""
    ist = pytz.timezone('Asia/Kolkata')
    now_ist = datetime.datetime.now(ist).time()
    start_time = datetime.time(18, 0, 0)  # 6:00 PM IST
    end_time = datetime.time(21, 0, 0)  # 8:00 PM IST
    return start_time <= now_ist <= end_time

def create_choropleth_map():
    """Creates an interactive Choropleth map with filters and highlights."""

    if not is_within_time_range():
        return None  

    
    filtered_df = merge_df[merge_df['Category'].str.startswith(('A', 'C', 'G', 'S'))]

    
    category_installs = filtered_df.groupby('Category')['Installs'].sum().sort_values(ascending=False).head(5)
    top_categories = category_installs.index.tolist()

   
    filtered_df = filtered_df[filtered_df['Category'].isin(top_categories)]

    
    highlighted_categories = category_installs[category_installs > 1000000].index.tolist()

    fig2 = go.Figure()

    for category in top_categories:
        category_data = filtered_df[filtered_df['Category'] == category]
        fig2.add_trace(go.Choropleth(
            locations=category_data['Country'],
            locationmode='country names',
            z=category_data['Installs'],
            colorscale='Viridis',
            colorbar_title='Installs',
            name=category,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            hovertemplate='<b>%{location}</b><br>Installs: %{z}<extra></extra>'
        ))

    fig2.update_layout(
        title_text='Global App Installs by Category (Top 5)',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'),
        
        margin=dict(l=20, r=20, t=50, b=20),
        legend_title_text='Categories',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    
    for trace in fig2.data:
        
        if trace.name in highlighted_categories:
            
            trace.name = f"<b>{trace.name}</b>"#bold the category name

    return fig2

 
map_fig2= create_choropleth_map()



In [43]:
map_fig2.update_traces(marker=dict(line=dict(color='black',width=4)))  
save_plot_as_html(map_fig2,"country.html", "Country wise analysis")

In [44]:
import plotly.graph_objects as go
import datetime

In [45]:
df=pd.read_csv('play_store_data.csv')

In [46]:
def create_conditional_violin_plot(df):
        now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=5, minutes=30)))
        if now.hour < 16 or now.hour >= 22:
            return None
        try:
            df_filtered_name = df[df['App'].str.contains('C', case=False, na=False)].copy()  
            
            df_filtered_rating = df_filtered_name[(df_filtered_name['Rating'] < 4.0) & (df_filtered_name['Reviews'] >= 10)].copy() 
            category_counts = df_filtered_rating['Category'].value_counts()  
            
            valid_categories = category_counts[category_counts > 50].index  # Filter categories with more than 50 apps
            df_filtered_category = df_filtered_rating[df_filtered_rating['Category'].isin(valid_categories)].copy()
            
            if df_filtered_category.empty:
                 return None
            fig= go.Figure()
            for category in valid_categories:
                category_data = df_filtered_category[df_filtered_category['Category'] == category]['Rating']
                fig.add_trace(go.Violin(x=[category] * len(category_data), y=category_data, name=category, box_visible=True, meanline_visible=True))
            
            fig.update_layout(
            title="Distribution of App Ratings (< 4.0) by Category (Apps with 'C' in Name, >50 Apps/Category, >10 Reviews)",
            xaxis_title="Category",
            yaxis_title="Rating",
            template="plotly_white",
            plot_bgcolor='black',
            paper_bgcolor='black',
            font_color='white',
        )
            return fig
            
        except (KeyError, TypeError, AttributeError) as e:
            print(f"Error creating violin plot: {e}")
            return None

In [47]:
try:
    df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce') # convert reviews to numeric
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce') #convert rating to numeric
except FileNotFoundError:
    print("Error: googleplaystore.csv not found.")
    df = pd.DataFrame() 

plot = create_conditional_violin_plot(df)



In [48]:
plot.update_traces(marker=dict(line=dict(color='white',width=4)))  
save_plot_as_html(plot,"Distribution of app ratings.html", "Distribution of app ratings")

In [49]:
dashboard_html="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name=viewport "content="width=device.width, initial-scale-1.0>
    <title> Google Play Store Analytics </title>
    <style>
        body{{
            font-family: Arial,sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            pading: 0;
            }}
        .header{{
            display: lex;
            align-items: center;
            justify-content: center;
            pading: 20px;
            background-color: #444
            }}
        .header img{{
            margin: 0 10px;
            height: 50px;
        }}
        .container{{
            display: flex;
            flex-wrap: wrap;
            justify_content: center;
            padding: 10px;
        }}
        .plot-container{{
            border: 2px solid #555
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: hidden;
            position: relative;
            cursor: pointer;
        }}
        .insights{{
            display: none;
            position: absolute;
            right: 10px;
            top: 10px;
            background-color: rgba(0,0,0,0,7);
            padding: 5px;
            border-radius: 5px;
            color:#fff;
            }}
        .plot-container:hover.insights{{
            display: block;
        }}
        </style>
        <script>
            function openPlot(filename){{
                window.open(filename,'_blank');
            }}
        </script>
</head>
<body>
    <div class="header">
        <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRH23YRx4BcLwvEWdfS_oxnt4zIi85DWrgbEQ&s">
         <h1>Google Play Store Analytics<h1>
          <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Google_2015_logo.svg/1200px-Google_2015_logo.svg.png">

    </div>
    <div class="container">
        {plots}
    </div>
</body>
</html>
"""

In [50]:
final_html=dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)

In [51]:
dashboard_path=os.path.join(html_files_path, "web page.html")

In [52]:
with open(dashboard_path,"w",encoding="UTF-8") as f:
    f.write(final_html)

In [53]:
webbrowser.open('file://'+os.path.realpath(dashboard_path))

True